In [1]:
import numpy as np
from nirs_data_class import nirs_data
import matplotlib.pyplot as plt
import filter as ft
from GLM_NIRS import GLM_NIRS
from visbrain.gui import Brain
from visbrain.objects import BrainObj, SourceObj, ConnectObj, ColorbarObj, SceneObj
from spatial_coordinates import read_coordinates

/home/ap/miniconda3/envs/NIRStoolkit/lib/python3.7/site-packages/nilearn/glm/__init__.py:56: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  'It may change in any future release of Nilearn.', FutureWarning)
/home/ap/miniconda3/envs/NIRStoolkit/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/home/ap/miniconda3/envs/NIRStoolkit/lib/python3.7/site-packages/vispy/visuals/line/line.py:395: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  ('color', np.float32, 4)])
/home/ap/miniconda3/envs/NIRStoolkit/lib/python3.7/site-packages/vispy/visuals/line/arrow.py:57: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of n

In [2]:
filepath = "/mnt/data/NIRS/data/sub-005/sub-005.nirs"

data = nirs_data()
data.default_labels['src'] = ''
data.default_labels['det'] = ''
data.read_homer2(filepath=filepath)
data.paradigm_make_block_design(block_length=20, conditions=[0])

INFO:nirs_data:Processing nirs file: /mnt/data/NIRS/data/sub-005/sub-005.nirs
INFO:nirs_data:16 sources, 16 detectors found
INFO:nirs_data:Sampling frequency: 7.813523650419287 Hz
INFO:nirs_data:Absorption coefficients loaded from: extinction_coeff.txt
INFO:nirs_data:Delta OD successfully converted to O2Hb and HHb concentraion changes. 41 channels, 7633 timepoints loaded
INFO:nirs_data:Recording time: 976.90 s


In [3]:
time_series = ft.butter_bandpass_filter(data.deoxyChannels, cut_low=0.01, cut_high=0.5, fs=data.fs, order=5)
time_series.shape

(41, 7633)

In [4]:
locations, loc_labels, src_coord, rec_coord, ref_coord = read_coordinates('/mnt/data/data/1219denoising/2020-01-20/positions.txt')
data.update_spatial_coordinates(src_coord, rec_coord)


In [5]:
glm = GLM_NIRS()
time_series.shape
glm.fs = data.fs
glm.create_design_matrix(data.trigger_block[:,0], np.atleast_2d(time_series[(20, 40), :]), first_derivative=False, sec_derivative=False)
       

In [6]:
 glm.show_design()

In [7]:
result_glm = glm.fit(np.delete(time_series, (20, 40), axis=0))
result_pval = glm.result_pval
result_beta = np.array(result_glm[0][:, 0]).astype(float)
result_ttest = np.array(result_glm[2]).astype(float)

Deg of freedom:136


/home/ap/miniconda3/envs/NIRStoolkit/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:541: FutureWarning: fft=True will become the default in a future version of statsmodels. To suppress this warning, explicitly set fft=False.
  warnings.warn(msg, FutureWarning)


In [8]:
[ch for ch in data.chLabels[data.short_channels_ind]]
data.remove_channels(data.short_channels_ind)

INFO:nirs_data:Channel removal complete. 2 channels removed.


In [9]:
order = np.array([ 0, 1,  4,  5,  6,  7,  8,  9, 13, 14, 15, 16, 17, 31, 32, 33, 34, 36, 37, 38, 
                  22, 23, 24, 25, 26, 27, 28, 29, 30, 35,  2,  3, 10, 11, 12, 18, 19, 20, 21])
colors = ['xkcd:violet', 'xkcd:violet', 'xkcd:violet', 'xkcd:violet', 'xkcd:periwinkle', 'xkcd:periwinkle', 
          'xkcd:periwinkle', 'xkcd:periwinkle', 'xkcd:periwinkle', 'xkcd:lavender', 'xkcd:lavender', 
          'xkcd:lavender', 'xkcd:lavender', 'xkcd:deep green', 'xkcd:deep green', 'xkcd:deep green', 
          'xkcd:deep green', 'xkcd:deep green', 'xkcd:deep green', 'xkcd:deep green', 'xkcd:leafy green', 
          'xkcd:leafy green', 'xkcd:leafy green', 'xkcd:leafy green', 'xkcd:leafy green', 'xkcd:leafy green', 
          'xkcd:leafy green', 'xkcd:leafy green', 'xkcd:leafy green', 'xkcd:leafy green', 'xkcd:deep blue', 
          'xkcd:deep blue', 'xkcd:cornflower', 'xkcd:cornflower', 'xkcd:cornflower', 'xkcd:light blue', 
          'xkcd:light blue', 'xkcd:marigold', 'xkcd:marigold'] 
legend = {'Frontal left': 'xkcd:violet', 'Frontal center': 'xkcd:periwinkle', 'Frontal right': 'xkcd:lavender', 
          'Motor left': 'xkcd:deep green', 'Motor center': 'xkcd:leafy green', 'Prefrontal left': 
          'xkcd:deep blue', 'Prefrontal center': 'xkcd:cornflower', 'Prefrontal right': 'xkcd:light blue', 
          'Temporal left': 'xkcd:marigold'}


In [10]:
glm.show_results_bar(mode ="t-test", order=order, colors=colors, 
                     labels=np.array(data.chLabels), color_labels=legend)

In [11]:
glm.show_results_bar(mode ="beta", order=order, colors=colors, 
                     labels=np.array(data.chLabels), color_labels=legend)

In [12]:
import matplotlib
matplotlib.use("Qt5Agg")

array([-0.24785438, -1.69355147, -1.36427416, -1.82416713, -1.08100761,
       -1.92670282, -1.24731691, -0.90206448, -0.73909181, -0.53840687,
       -1.86596742, -1.32314275, -0.45293242, -1.40000242, -1.0644785 ,
       -0.81621735, -0.10086666,  0.38736653,  2.06610299,  0.44052181,
        1.66178015,  0.34719125, -0.12229456, -1.06169018, -1.0653156 ,
        1.39999954, -0.26045973,  0.35428903,  0.31146354, -0.07435615,
        1.25888792,  5.06512015, -0.66059901,  0.53865694,  1.15903625,
        5.21622564,  9.01307067,  3.19845116,  1.84492727])

In [15]:
sc = SceneObj(size=(1600, 1000))
kwargs = {}
kwargs['color'] = "magenta"
kwargs['alpha'] = 0.7
kwargs['data'] = result_ttest
kwargs['radius_min'] = 20               # Minimum radius
kwargs['radius_max'] = 21              # Maximum radius
kwargs['edge_color'] = (1, 1, 1, 0.5)  # Color of the edges
kwargs['edge_width'] = .5              # Width of the edges
kwargs['symbol'] = 'o'            # Source's symbol
kwargs['text_size'] = 10.0
kwargs['text'] = data.chLabels
kwargs['clim'] = (-10, 10)
cb_kw = dict(cblabel="Task related activation", cbtxtsz=3., border=False, cmap='seismic',
             clim=(-10, 10))

xyz = data.xyz

Creation of a scene
INFO:visbrain:Creation of a scene


In [16]:
s_obj = SourceObj('Sobj', xyz, **kwargs)
s_obj.color_sources(data=result_ttest, cmap='seismic')

b_obj = BrainObj('B2', **cb_kw)

b_obj_vert = b_obj.vertices
s_obj.fit_to_vertices(b_obj_vert)

fitted_coord = s_obj._sources._data['a_position']
s1_obj = SourceObj('Sobj', fitted_coord * 0.95, **kwargs)
s1_obj.color_sources(data=result_ttest, cmap='seismic')
s1_obj.project_sources(b_obj, cmap='seismic', clim=(-10, 10))

vb = Brain(source_obj=s1_obj, brain_obj=b_obj, bgcolor='lightblue') #connect_obj=c_obj,
vb.brain_control(translucent=False)
vb.menuDispCbar.setChecked(True)
vb.rotate('axial_0')
vb.show()

SourceObj(name='Sobj') created
INFO:visbrain:SourceObj(name='Sobj') created
    39 sources detected
INFO:visbrain:    39 sources detected
/home/ap/miniconda3/envs/NIRStoolkit/lib/python3.7/site-packages/vispy/visuals/markers.py:560: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  ('a_edgewidth', np.float32, 1)])
    Color Sobj using a data vector
INFO:visbrain:    Color Sobj using a data vector
BrainObj(name='B2') created
INFO:visbrain:BrainObj(name='B2') created
/home/ap/miniconda3/envs/NIRStoolkit/lib/python3.7/site-packages/vispy/gloo/buffer.py:454: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  data = data.view(dtype=[('f0', data.dtype.base, c)])
SourceObj(name='Sobj') created
INFO:visbrain:SourceObj(name='Sobj') created
    39 sources detected
INFO:vis